# PyCity Schools Analysis - Summary
**Data Analyst: Jeff Brown**

## Analysis of District-level Performance

The PyCity School District serves over 39,000 students through the efforts of
teachers and administrators at 15 District and Charter schools, with a total budget of $24.6 million.

* The average district student exceeds minimum performance expectatations (passing: math and reading scores >= 70)
* However, the % Passing Both Math and Reading (65.17%) metric suggests that on average almost 35% of district students fail to pass either Math or Reading, which should be a source of concern to the district.

* While there is some range of performance on average student Reading scores on a per-school basis, the variation in math scores is more pronounced:
    * Average Math Scores: 76.6 to 83.8 (7.2 percentage point difference) vs. Average Reading Scores: 80.7 to 84.0 (only 3.3 percentage point difference)
    * % Passing Math: 65.7% to 94.6% (28.9 percentage point difference)
    * % Passing Both Math and Reading: 53.0% to 91.3% (**38.3** percentage point difference!)

* The metrics analyzed below indicate that student math scores vary more significantly with school size and school type, and show less variation with school spending and student grade level:
    * % Passing Math is 70.0% at Large size schools vs. 93.6% at both Medium and Small size schools

* As a next, analysis of additional school data may help determine if the variation with School Type is due to the relative size of Charter vs District schools, or if there are other factors involved:
    * Charter schools (average = 1524 students) on average have less than half the number of students as District schools (average = 3854 students)


# Manage Dependencies and Gather Relevant Data

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

## Read School Data into a Dataframe

In [2]:
# Read School Data File and store into Pandas Data Frames
school_data_to_load = "Resources/schools_complete.csv"
sch_d = pd.read_csv(school_data_to_load)
sch_d.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


## Read Student Data into a Dataframe

In [3]:
# Read Student Data File and store into Pandas Data Frames
student_data_to_load = "Resources/students_complete.csv"
stu_d = pd.read_csv(student_data_to_load)
stu_d.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## Consolidate School and Student Data into a Single Dataframe

In [4]:
# Combine the data into a single dataset
full_d = pd.merge(stu_d, sch_d, how="left", left_on="school_name", right_on="school_name")
full_d.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

# District-level Performance


## Basic District-level Metrics

In [5]:
# Use agg or apply methods to calculate needed statistics
# School data statistics:
#  total number of schools
#  total budget
tot_n_sch, tot_budget = sch_d.agg({"school_name":"count",
                                   "budget":"sum"
                                  })

In [6]:
# Use agg or apply methods to calculate needed statistics
# Student data statistics
#  total number of students
#  average math score
#  average reading score
tot_n_stu, avg_math_sc, avg_read_sc = stu_d.agg({"student_name":"count",
                                                 "math_score":"mean",
                                                 "reading_score":"mean"
                                                })

#  overall average score, i.e. (avg. math score + avg. reading score)/2
avg_overall_sc = (avg_math_sc + avg_read_sc)/2

In [7]:
# @TODO: Define function to calculate pass rate (vs. repeated code) => DONE!
#
# Function: Calculate Passing Rate for 1 or 2 sets of scores
#
# Determine if each student passed (>=70) or failed (<70) up to two subjects
# Then use this info to count "Pass" / Total Students
#
# Arguments:
#   a = Series containing scores to be compared with passing threshold (70.0)
#   b = Optional Series containing scores to be compared with passing threshold (70.0)
#
# Return value:
#   retval = Scalar value representing the pass rate of students in the Series

def p_rate(a, b = None):
    if (b is None):
        p_sc = (a >= 70.0)
    else:
        p_sc = (a >= 70.0) & (b >= 70.0)

    retval = p_sc[ p_sc == True ].count() / p_sc.count()
    
    return retval

### Pass Rate Metrics
The requested pass rate metrics are:
* **% Passing Math**: Percentage of students with a passing math score (70 or greater)
* **% Passing Reading**: Percentage of students with a passing math score (70 or greater)
* **% Passing Overall**: The average of the % Passing Math and % Passing Reading scores

In addition, I have chosen to also calculate:
* **% Passing Both Math and Reading**: Percentage of students with a passing score in both math _and_ reading (i.e., 70 or greater in both subjects)

The standard of success for students should be passing scores in both of these core, fundamental subjects.
It will also be interesting to see if this % Passing Both metric provides a different view of district or school performance vs. the requested % Passing Overall metric.  The District, School Summary, and other various reports include this metrics.

In [8]:
# Use agg or apply methods to calculate needed statistics
# Student data statistics
#  percentage of students with a passing math score (70 or greater)
# OLD APPROACH: pass_math = stu_d["student_name"].loc[ stu_d["math_score"] >= 70.0 ].count() / tot_n_stu
pass_math = p_rate( stu_d['math_score'] )

#  percentage of students with a passing reading score (70 or greater)
pass_reading = p_rate(stu_d['reading_score'])

#  percentage of students with overall passing score (average of % passing math and reading)
pass_overall = (pass_math + pass_reading)/2

#  percentage of students with a passing both math and reading (both math and reading 70 or greater)
# OLD APPROACH: pass_both = stu_d["student_name"].loc[ ( (stu_d["math_score"] >= 70.0) & (stu_d["reading_score"] >= 70.0) ) ].count() / tot_n_stu
pass_both = p_rate( stu_d['math_score'], stu_d['reading_score'])

In [ ]:
district_summary = pd.DataFrame( {"Total Schools": [ f"{tot_n_sch:,.0f}" ],
                                  "Total Students": [ f"{tot_n_stu:,.0f}" ],
                                  
                                  "Total Budget": [ f"${tot_budget:,.2f}" ],
                                  
                                  "Average Math Score": [ f"{avg_math_sc:.1f}" ],
                                  "Average Reading Score": [ f"{avg_read_sc:.1f}" ],
                                  "Average Overall Score": [ f"{avg_overall_sc:.1f}" ],
                                  
                                  "% Passing Math": [ f"{pass_math:.2%}" ],
                                  "% Passing Reading": [ f"{pass_reading:.2%}" ],
                                  "% Passing Overall": [ f"{pass_overall:.2%}" ],               
                                  "% Passing Both Math and Reading": [ f"{pass_both:.2%}" ]                
                                 })

## Analysis of District-level Performance

The PyCity School District serves over 39,000 students through the efforts of
teachers and administrators at 15 District and Charter schools, with a total budget of $24.6 million.
* Average Math (79.0) and Reading (81.9) scores are above 70 (passing), indicating the *average district student* exceeds minimum performance expectatations (passing: score >= 70)
* This is reflected in the results of Average % Passing Math (74.98%) and Reading (85.81%) metrics
* However, the % Passing Both Math and Reading (65.17%) metric suggests that on average almost 35% of district students fail to pass either Math or Reading, which should be a source of concern to the district.
* Additional analysis below will examine these metrics on a per-School basis, by School size, spending, and school type to determine if there is a relationship between student performance and these other factors.


In [45]:
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
0,15,"39,170","$24,649,428.00",79.0,81.9,80.4,74.98%,85.81%,80.39%,65.17%


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


# School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

# School-level Performance


## Basic School-level Metrics

In [11]:
full_d.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [12]:
# Group by 'school_name'
sch_g = full_d.groupby('school_name')

In [13]:
# Calculate per School values for:
#   School Name -- already the key for the dataframe
#   School Type
#   Total Students
#   Total School Budget
sch_s_df = sch_g.agg({"type":"max",
                      "student_name":"count",
                      "budget":"max"
                     })

# Rename the columns in the School Summary dataframe
sch_s_df.rename(inplace=True, columns={"type":"School Type",
                         "student_name":"Total Students",
                         "budget":"Total Budget"
                        })

In [14]:
# Enter per-School values into the School summary dataframe
#   Per Student Budget
sch_s_df['Budget per Student'] = sch_s_df['Total Budget'] / sch_s_df['Total Students']

In [15]:
#Average Math Score
#Average Reading Score
#Average Oveall Score
sch_s_df['Average Math Score'] = sch_g.agg({ "math_score":"mean" })
sch_s_df['Average Reading Score'] = sch_g.agg({"reading_score":"mean"})
sch_s_df['Average Overall Score'] = (sch_s_df['Average Math Score'] + sch_s_df['Average Reading Score'])/2

In [16]:
# Calculate:
#   % Passing Math
#   % Passing Reading
#   Overall Passing Rate (Average of the above two)

#  percentage of students with a passing math score (70 or greater)
sch_s_df['% Passing Math'] = sch_g.apply(lambda x: p_rate(x.math_score) )

#  percentage of students with a passing reading score (70 or greater)
sch_s_df['% Passing Reading'] = sch_g.apply(lambda x: p_rate(x.reading_score) )

#  percentage of students with an overall passing score (average of % pass Math and Reading)
sch_s_df['% Passing Overall'] = (sch_s_df['% Passing Math'] + sch_s_df['% Passing Reading'])/2.0

In [46]:
#Calculate
# percentage of students with a passing both math and reading (both math and reading 70 or greater)
sch_s_df['% Passing Both Math and Reading'] = sch_g.apply(lambda x: p_rate(x.math_score, x.reading_score) )


In [47]:
# @TODO: Make a new DataFrame with data formatted to match the District Summary => DONE!
school_summary_df = pd.DataFrame(index = sch_s_df.index )

school_summary_df['School Type'] = sch_s_df['School Type'].apply('{:}'.format) 

school_summary_df['Total Students'] = sch_s_df['Total Students'].apply('{:,.0f}'.format) 

school_summary_df['Total Budget'] = sch_s_df['Total Budget'].apply('{:,.2f}'.format) 
school_summary_df['Budget per Student'] = sch_s_df['Budget per Student'].apply('{:,.2f}'.format) 

school_summary_df['Average Math Score'] = sch_s_df['Average Math Score'].apply('{:.1f}'.format) 
school_summary_df['Average Reading Score'] = sch_s_df['Average Reading Score'].apply('{:.1f}'.format) 
school_summary_df['Average Overall Score'] = sch_s_df['Average Overall Score'].apply('{:.1f}'.format) 

school_summary_df['% Passing Math'] = sch_s_df['% Passing Math'].apply('{:.2%}'.format) 
school_summary_df['% Passing Reading'] = sch_s_df['% Passing Reading'].apply('{:.2%}'.format) 
school_summary_df['% Passing Overall'] = sch_s_df['% Passing Overall'].apply('{:.2%}'.format) 
school_summary_df['% Passing Both Math and Reading'] = sch_s_df['% Passing Both Math and Reading'].apply('{:.2%}'.format) 

## Analysis of School-level Performance

There is significant variation in performnce of the 15 schools that comprise the PyCity School District.
* While there is not much difference in Reading scores from school-to-school:
    * Average Reading Scores: 80.7 to 84.0 (only 3.3 percentage point difference)

* However, some per-School scores vary significantly:
    * Average Math Scores: 76.6 to 83.8 (7.2 percentage point difference)
    * % Passing Math: 65.7% to 94.6% (28.9 percentage point difference)
    * % Passing Reading: 79.3% to 97.3% (18 percentage point difference)
    * % Passing Both Math and Reading: 53.0% to 91.3% (**38.3** percentage point difference!)

In [54]:
sch_s_df.describe()

,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
count,15.000000,1.500000e+01,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,2611.333333,1.643295e+06,620.066667,80.432530,82.529188,81.480859,0.809871,0.892190,0.851030,0.732776
std,1420.915282,9.347763e+05,28.544368,3.374548,1.519329,2.442526,0.139988,0.081807,0.110716,0.189921
min,427.000000,2.480870e+05,578.000000,76.629414,80.744686,78.793698,0.656839,0.792990,0.732933,0.529882
25%,1698.000000,1.046265e+06,591.500000,77.060448,81.000178,78.977161,0.665233,0.810427,0.737221,0.535333
50%,2283.000000,1.319574e+06,628.000000,83.061895,83.725724,83.518837,0.925059,0.958546,0.943794,0.892272
75%,3474.000000,2.228999e+06,641.500000,83.388902,83.901965,83.632742,0.936297,0.965754,0.952347,0.905616
max,4976.000000,3.124928e+06,655.000000,83.839917,84.044699,83.942308,0.945946,0.973089,0.955867,0.913348


In [49]:
school_summary_df

,School Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
school_name,,,,,,,,,,,
Bailey High School,District,"4,976","3,124,928.00",628.00,77.0,81.0,79.0,66.68%,81.93%,74.31%,54.64%
Cabrera High School,Charter,"1,858","1,081,356.00",582.00,83.1,84.0,83.5,94.13%,97.04%,95.59%,91.33%
Figueroa High School,District,"2,949","1,884,411.00",639.00,76.7,81.2,78.9,65.99%,80.74%,73.36%,53.20%
Ford High School,District,"2,739","1,763,916.00",644.00,77.1,80.7,78.9,68.31%,79.30%,73.80%,54.29%
Griffin High School,Charter,"1,468","917,500.00",625.00,83.4,83.8,83.6,93.39%,97.14%,95.27%,90.60%
Hernandez High School,District,"4,635","3,022,020.00",652.00,77.3,80.9,79.1,66.75%,80.86%,73.81%,53.53%
Holden High School,Charter,427,"248,087.00",581.00,83.8,83.8,83.8,92.51%,96.25%,94.38%,89.23%
Huang High School,District,"2,917","1,910,635.00",655.00,76.6,81.2,78.9,65.68%,81.32%,73.50%,53.51%
Johnson High School,District,"4,761","3,094,650.00",650.00,77.1,81.0,79.0,66.06%,81.22%,73.64%,53.54%


# Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [19]:
# Top Performing Schools (By % Passing Overall)
# (Differs from ranking in the sample out only due to rounding to 2 decimal places)
school_summary_df.sort_values(by=['% Passing Overall'], ascending=False).head(5)

,School Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
school_name,,,,,,,,,,,
Cabrera High School,Charter,"1,858","1,081,356.00",582.00,83.1,84.0,83.5,94.13%,97.04%,95.59%,91.33%
Thomas High School,Charter,"1,635","1,043,130.00",638.00,83.4,83.8,83.6,93.27%,97.31%,95.29%,90.95%
Griffin High School,Charter,"1,468","917,500.00",625.00,83.4,83.8,83.6,93.39%,97.14%,95.27%,90.60%
Pena High School,Charter,962,"585,858.00",609.00,83.8,84.0,83.9,94.59%,95.95%,95.27%,90.54%
Wilson High School,Charter,"2,283","1,319,574.00",578.00,83.3,84.0,83.6,93.87%,96.54%,95.20%,90.58%


*[NOTE: Woops, lost the sample output that was here by accidently rerunning this cell]*

In [20]:
# Top Performing Schools (By % Passing Both Math and Reading)
school_summary_df.sort_values(by=['% Passing Both Math and Reading'], ascending=False).head(5)

,School Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
school_name,,,,,,,,,,,
Cabrera High School,Charter,"1,858","1,081,356.00",582.00,83.1,84.0,83.5,94.13%,97.04%,95.59%,91.33%
Thomas High School,Charter,"1,635","1,043,130.00",638.00,83.4,83.8,83.6,93.27%,97.31%,95.29%,90.95%
Griffin High School,Charter,"1,468","917,500.00",625.00,83.4,83.8,83.6,93.39%,97.14%,95.27%,90.60%
Wilson High School,Charter,"2,283","1,319,574.00",578.00,83.3,84.0,83.6,93.87%,96.54%,95.20%,90.58%
Pena High School,Charter,962,"585,858.00",609.00,83.8,84.0,83.9,94.59%,95.95%,95.27%,90.54%


# Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [21]:
# Bottom Performing Schools (By % Passing Overall)
school_summary_df.sort_values(by=['% Passing Overall'], ascending=True).head(5)

,School Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
school_name,,,,,,,,,,,
Rodriguez High School,District,"3,999","2,547,363.00",637.00,76.8,80.7,78.8,66.37%,80.22%,73.29%,52.99%
Figueroa High School,District,"2,949","1,884,411.00",639.00,76.7,81.2,78.9,65.99%,80.74%,73.36%,53.20%
Huang High School,District,"2,917","1,910,635.00",655.00,76.6,81.2,78.9,65.68%,81.32%,73.50%,53.51%
Johnson High School,District,"4,761","3,094,650.00",650.00,77.1,81.0,79.0,66.06%,81.22%,73.64%,53.54%
Ford High School,District,"2,739","1,763,916.00",644.00,77.1,80.7,78.9,68.31%,79.30%,73.80%,54.29%


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [23]:
# Bottom Performing Schools (By % Passing Both Math and Reading)
school_summary_df.sort_values(by=['% Passing Both Math and Reading'], ascending=True).head(5)

,School Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,Average Overall Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
school_name,,,,,,,,,,,
Rodriguez High School,District,"3,999","2,547,363.00",637.00,76.8,80.7,78.8,66.37%,80.22%,73.29%,52.99%
Figueroa High School,District,"2,949","1,884,411.00",639.00,76.7,81.2,78.9,65.99%,80.74%,73.36%,53.20%
Huang High School,District,"2,917","1,910,635.00",655.00,76.6,81.2,78.9,65.68%,81.32%,73.50%,53.51%
Hernandez High School,District,"4,635","3,022,020.00",652.00,77.3,80.9,79.1,66.75%,80.86%,73.81%,53.53%
Johnson High School,District,"4,761","3,094,650.00",650.00,77.1,81.0,79.0,66.06%,81.22%,73.64%,53.54%


# Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
# Create a dataframe to hold Math Summary
# Use the indices from the School Summary dataframe
math_summary = pd.DataFrame( index=school_summary_df.index )

# Loop through all of the grade levels and build the dataframe
for g_level in [ '9th', '10th', '11th', '12th']:
    math_summary[g_level] = full_d.loc[ full_d['grade'] == g_level ].groupby('school_name').agg({"math_score":"mean"})

## Analysis of School Performance per-Grade based upon Math Scores

The average math scores of students in the district does not seem to vary significantly based upon the grade level (9th, 10th, 11th, 12th) level of the students, as can be seen from the Mean, Minimum, and Maximum metrics noted below.

In [52]:
math_summary.describe()

,9th,10th,11th,12th
count,15.000000,15.000000,15.000000,15.000000
mean,80.351617,80.378948,80.575873,80.423811
std,3.226342,3.479296,3.538353,3.372793
min,76.403037,75.908735,76.395626,76.179963
25%,77.135767,76.843944,77.027044,77.168968
50%,82.044010,82.917411,82.765560,82.855422
75%,83.342730,83.400912,83.667788,83.426603
max,83.787402,84.229064,85.000000,84.121547


In [24]:
math_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
# Create a dataframe to hold Math Summary
# Use the indices from the School Summary dataframe
reading_summary = pd.DataFrame( index=school_summary_df.index )

# Loop through all of the grade levels and build the dataframe
for g_level in [ '9th', '10th', '11th', '12th']:
    reading_summary[g_level] = full_d.loc[ full_d['grade'] == g_level ].groupby('school_name').agg({"reading_score":"mean"})

## Analysis of School Performance per-Grade based upon Reading Scores

The average reading scores of students in the district does not seem to vary significantly based upon the grade level (9th, 10th, 11th, 12th) level of the students, as can be seen from the Mean, Minimum, and Maximum metrics noted below.

In [55]:
reading_summary.describe()

,9th,10th,11th,12th
count,15.000000,15.000000,15.000000,15.000000
mean,82.513318,82.505439,82.559485,82.554817
std,1.408513,1.477121,1.643606,1.753843
min,80.632653,80.629808,80.403642,80.305983
25%,81.229656,81.084948,80.905227,80.884797
50%,83.369193,83.324561,83.585542,82.781671
75%,83.768062,83.759827,83.985928,84.180564
max,84.122642,84.254157,84.373786,84.698795


In [25]:
reading_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [26]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [28]:
# Get the relevant data, including the columns that will be needed below
gen_s_df = sch_s_df[['School Type', 'Total Students', 'Total Budget', 
          'Average Math Score', 'Average Reading Score',
          '% Passing Math', '% Passing Reading','% Passing Overall',
          '% Passing Both Math and Reading']]

In [30]:
# Calculate the budget per student and add to dataframe
gen_s_df['Budget per Student'] = gen_s_df['Total Budget'] / gen_s_df['Total Students']

C:\Users\dadja\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
# Scores by School Type
gen_s_df['Spending Ranges (Per Student)'] = pd.cut(gen_s_df["Budget per Student"], spending_bins, labels=group_names)

In [32]:
scores_by_spending_df = gen_s_df.groupby('Spending Ranges (Per Student)').agg(np.mean)

In [34]:
scores_by_spending_df = scores_by_spending_df[[
          'Average Math Score', 'Average Reading Score',
          '% Passing Math', '% Passing Reading','% Passing Overall',
          '% Passing Both Math and Reading']]

In [35]:
# Function: formatscores
# Apply some basic formating to average and percentage scores
def formatscores(a_df):
    a_df['Average Math Score'] = a_df['Average Math Score'].apply('{:.1f}'.format) 
    a_df['Average Reading Score'] = a_df['Average Reading Score'].apply('{:.1f}'.format) 

    a_df['% Passing Math'] = a_df['% Passing Math'].apply('{:.2%}'.format) 
    a_df['% Passing Reading'] = a_df['% Passing Reading'].apply('{:.2%}'.format) 
    a_df['% Passing Overall'] = a_df['% Passing Overall'].apply('{:.2%}'.format) 
    a_df['% Passing Both Math and Reading'] = a_df['% Passing Both Math and Reading'].apply('{:.2%}'.format) 


In [ ]:
# Use formatscores function to format average and percent score columns
formatscores(scores_by_spending_df)

## Analysis of School Performance by Spending

While it might be expected (and even hoped for) that increasing Spending per Student would increase student performance, the analysis show the opposite trend for the performance metrics being analyzed:

* Students at schools with the lowest level of spending (less than $585 per student) had the highest overall performance across most metrics

* Students at schools with the highest level of spending ($645 to $675 per student) had the lower overall performance across the metrics

* While this could lead to a conclusion that reducing school funding will lead to higher levels of student performance, it might be more prudent to consider other factors affecting student performance that also are more prevalent at schools with higher funding

In [36]:
scores_by_spending_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
Spending Ranges (Per Student),,,,,,
<$585,83.5,83.9,93.46%,96.61%,95.04%,90.37%
$585-615,83.6,83.9,94.23%,95.90%,95.07%,90.22%
$615-645,79.1,81.9,75.67%,86.11%,80.89%,66.11%
$645-675,77.0,81.0,66.16%,81.13%,73.65%,53.53%


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


# Scores by School Size

* Perform the same operations as above, based on school size.

In [37]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [38]:
# Scores by School Size
gen_s_df['School Size'] = pd.cut(gen_s_df["Total Students"], size_bins, labels=group_names)

In [39]:
# Group by School Size
scores_by_size_df = gen_s_df.groupby('School Size').agg(np.mean)

In [40]:
# Keep just the columns needed in the summary table
scores_by_size_df = scores_by_size_df[[
          'Average Math Score', 'Average Reading Score',
          '% Passing Math', '% Passing Reading','% Passing Overall',
          '% Passing Both Math and Reading']]

In [ ]:
# Use formatscores function to format average and percent score columns
formatscores(scores_by_size_df)

## Analysis of School Performance by School Size

The analysis shows that students at Medium (1000-2000) and Small (<1000) size schools outperform students at Large (2000-5000) size schools based upon the metrics analyzed:

* It's particularly noteworthy that about 42.7% of students at Large size schools fail to pass either Math or Reading, while less than 10% of students fall into this category at Medium or Small size schools.
* While there is some variation in % Passing Reading, the major impact is due to % Passing Math, which is 70.0% at Large size schools vs. 93.6% at both Medium and Small size schools

In [57]:
scores_by_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
School Size,,,,,,
Small (<1000),83.8,83.9,93.55%,96.10%,94.82%,89.88%
Medium (1000-2000),83.4,83.9,93.60%,96.79%,95.20%,90.62%
Large (2000-5000),77.7,81.3,69.96%,82.77%,76.36%,58.29%


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


# Scores by School Type

* Perform the same operations as above, based on school type.

In [64]:
# Scores by School Type
scores_by_type_df = sch_s_df[['School Type','Total Students',
          'Average Math Score', 'Average Reading Score',
          '% Passing Math', '% Passing Reading','% Passing Overall',
          '% Passing Both Math and Reading']].groupby('School Type').agg(np.mean).head()

# Use formatscores function to format average and percent score columns
formatscores(scores_by_type_df)

## Analysis of School Performance by School Type

The analysis shows that students at Charter schools outperform students at District schools based upon the metrics analyzed:
* This result seem to align with the analysis of School Performance by School Size, since Charter schools (average = 1524 students) on average have less than half the number of students as District schools (average = 3854 students)

In [66]:
scores_by_type_df

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both Math and Reading
School Type,,,,,,,
Charter,1524.250000,83.5,83.9,93.62%,96.59%,95.10%,90.43%
District,3853.714286,77.0,81.0,66.55%,80.80%,73.67%,53.67%


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
